In [4]:
import earthaccess
import xarray as xr
import rasterio as rio
import glob2
from rasterio.transform import from_origin
import geopandas as geo
import numpy as np
import os
from rasterio.mask import mask

c:\Users\bzwil\anaconda3\envs\bzwdomain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# authenticate with Earthdata
earthaccess.login(strategy="netrc")

In [ ]:
# MOD13Q1 (vegetaion indices 16-day L3 global 250m)
granules = earthaccess.search_data(
    short_name = "MOD13Q1",
    temporal = ("2024-04-01", "2024-04-30"), 
    bounding_box = (19.8, 54.3, 21.5, 55.1), 
    cloud_hosted = True
)

download_files = earthaccess.download(granules[:1])
print(download_files)

# situate the ndvi file properly
ndvi_path = r'C:\Users\bzwil\OneDrive\Desktop\KaliningradProject\kaliningrad-geoint-change-detection\files\data\geotiffs\ndvi_output.tif'

with rio.open(ndvi_path) as ndvi:
    ndvi_data = ndvi.read(1).astype('float32')
    ndvi_data[ndvi_data == -3000] = float('nan')
    ndvi_data = ndvi_data / 10000.0 
    
    print("NDVI shape:", ndvi_data.shape)

In [ ]:
# web scraping DEM
# SRTMGL1 DEM tiles
granules = earthaccess.search_data(
    short_name = "SRTMGL1",
    temporal = ("2000-02-11", "2000-02-11"), 
    bounding_box = (19.8, 54.3, 21.5, 55.1), 
    cloud_hosted = True
)

# downloading first tile 
download_DEM = earthaccess.download(granules[:1])
dem_path = download_DEM[0]
print("downloaded DEM:", dem_path)

# load kaliningrad aoi & clip to dem
aoi_path = r"C:\Users\bzwil\OneDrive\Desktop\KaliningradProject\kaliningrad-geoint-change-detection\files\data\kaliningrad-oblast.geojson"
aoi = geo.read_file(aoi_path)

with rio.open(dem_path) as src:
    aoi = aoi.to_crs(src.crs)
    clipped_dem, transform = mask(src, aoi.geometry, crop=True)
    out_meta = {
    "driver": "GTiff",
    "dtype": clipped_dem.dtype,
    "count": 1,
    "crs": src.crs,
    "transform": transform,
    "width": int(clipped_dem.shape[2]),
    "height": int(clipped_dem.shape[1])
}

out_meta.update({
    "height": clipped_dem.shape[1],
    "width": clipped_dem.shape[2],
    "transform": transform
})

output_path = r"C:\Users\bzwil\OneDrive\Desktop\KaliningradProject\kaliningrad-geoint-change-detection\files\data\dem\kaliningrad_dem_clipped.tif"
with rio.open(output_path, "w", **out_meta) as dst:
    dst.write(clipped_dem)

In [ ]:
path = r"C:\Users\bzwil\OneDrive\Desktop\KaliningradProject\kaliningrad-geoint-change-detection\files\data\viirs_nc"

nc_files = glob.glob(os.path.join(path, "*.nc"))
target_variable = 'fire mask' # paramount data

# processing each .nc file & matching file shape
for nc_file in nc_files:
    try:
        ds = xr.open_dataset(nc_file)
        
        fire_mask = ds[target_variable].values  
        lat = ds['FP_latitude'].values         
        lon = ds['FP_longitude'].values       
        print("Variables extracted: fire mask, latitude, longitude", flush=True)

        if fire_mask.shape != lat.shape or fire_mask.shape != lon.shape:
            rows, cols = fire_mask.shape
        else:
            rows, cols = fire_mask.shape
        
        nodata_value = 255 if fire_mask.dtype == 'uint8' else -9999     
               
        lon_min, lon_max = lon.min(), lon.max()
        lat_min, lat_max = lat.min(), lat.max()
        rows, cols = fire_mask.shape
        x_res = (lon_max - lon_min) / cols
        y_res = (lat_max - lat_min) / rows

        transform = from_origin(lon_min, lat_max, x_res, y_res)

        metadata = {
            'driver': 'GTiff',
            'height': rows,
            'width': cols,
            'count': 1,  
            'dtype': fire_mask.dtype,
            'crs': 'EPSG:4326',  
            'transform': transform,
            'nodata': nodata_value  
        }

        # creating output filename
        output_tif = os.path.join(path, os.path.splitext(os.path.basename(nc_file))[0] + f"_{target_variable.replace(' ', '_')}.tif")
        print(f"Output file will be: {output_tif}", flush=True)

        # writing geotiff
        with rasterio.open(output_tif, 'w', **metadata) as dst:
            dst.write(fire_mask, 1)  # Write as band 1
            print(f"Saved GeoTIFF: {output_tif}", flush=True)

        ds.close()
    except Exception as e:
        print(f"Error processing {nc_file}: {e}", flush=True)
        continue

Script started!
Current working directory: C:\Users\bzwil\OneDrive\Desktop\KaliningradProject\kaliningrad-geoint-change-detection\files\data\viirs_nc
Found .nc files: ['C:\\Users\\bzwil\\OneDrive\\Desktop\\KaliningradProject\\kaliningrad-geoint-change-detection\\files\\data\\viirs_nc\\VNP14IMG.A2024092.0036.002.2024093145722.nc', 'C:\\Users\\bzwil\\OneDrive\\Desktop\\KaliningradProject\\kaliningrad-geoint-change-detection\\files\\data\\viirs_nc\\VNP14IMG.A2024092.0218.002.2024093145719.nc', 'C:\\Users\\bzwil\\OneDrive\\Desktop\\KaliningradProject\\kaliningrad-geoint-change-detection\\files\\data\\viirs_nc\\VNP14IMG.A2024092.1030.002.2024093145722.nc']
Processing C:\Users\bzwil\OneDrive\Desktop\KaliningradProject\kaliningrad-geoint-change-detection\files\data\viirs_nc\VNP14IMG.A2024092.0036.002.2024093145722.nc...
Dataset loaded successfully.
Variables extracted: fire mask, latitude, longitude
data type of fire mask: uint8
using nodata value: 255
Spatial bounds: lon(2.2693140506744385, 